### Load data

Choose that dataset to load. We have prepared 2 different datasets, downloaded and converted from the competition:
* wbg-fal10: dataset_01 ([ITC2019 description](https://www.itc2019.org/instances/status/5b59b12884ed6a262b1ce9d3))
* pu-cs-fal07: dataset_04 ([ITC2019 description](https://www.itc2019.org/instances/status/5b59ebf984ed6a262b1ce9dc))

Choose between datasets by adjusting the command `import dataset_0x as dataset`

In [32]:
import dataset_01 as dataset
from docplex.cp.model import CpoModel
import time

In [33]:

rooms = dataset.rooms
courses = dataset.courses
students = dataset.students
constraints = dataset.constraints

List of supporting tables:
* `classes`: 150 classes with details
* `classes_id`: ids from 1 to 150
* `rooms_id`: ids from 1 to 7
* `students_id`: ids from 1 to 19
* `student_subparts`: pairs of 1 student & 1 subpart that he/she must attend (the subpart is a list of classes_id)
* `student_classes`: pairs of 1 student & the list of all classes that he/she might attend (flattened from above subpart) -> 19 pairs

### Class - Time pairs

In [3]:
classes = []

for course in courses:
    for subpart in course['subpart']:
        for c in subpart['class']:
            classes.append(c)

classes

[{'id': 1,
  'limit': 2,
  'room': [{'id': 6, 'penalty': 4},
   {'id': 7, 'penalty': 0},
   {'id': 3, 'penalty': 0}],
  'time': [{'days': '1010100',
    'start': 102,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 16},
   {'days': '1010100',
    'start': 114,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 126,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 138,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 150,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 162,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 174,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 16},
   {'days': '1010100',
    'start': 186,
    'length': 10,
    'weeks': '1111111111111111',
   

In [4]:
classes_id = [c['id'] for c in classes]

### Class - Room pairs

In [5]:
rooms_id = [r['id'] for r in rooms]
rooms_id

[1, 2, 3, 4, 5, 6, 7]

### Student - Class pairs

In [6]:
students_id = [s['id'] for s in students]

In [7]:
student_subparts = []

for s in students_id:
    for i in students[s-1]['course']:
        for subpart in courses[i-1]['subpart']:
            student_subparts.append([s,[c['id'] for c in subpart['class']]])           
            
student_subparts

[[1, [1]],
 [1, [142]],
 [1, [143, 144]],
 [1, [79, 80, 81, 82, 83, 84, 85, 86, 87]],
 [1, [88]],
 [2, [1]],
 [2, [142]],
 [2, [79, 80, 81, 82, 83, 84, 85, 86, 87]],
 [2, [88]],
 [2, [113, 114]],
 [2, [115, 116, 117, 118]],
 [3, [142]],
 [3, [31, 32]],
 [3, [33, 34, 35, 36]],
 [3, [37, 38, 39]],
 [3, [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]],
 [3, [88]],
 [3, [89, 90, 91, 92, 93, 94, 95, 96, 97, 98]],
 [3, [99, 100, 101, 102, 103, 104, 105, 106, 107, 108]],
 [4, [3, 4, 5]],
 [4, [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]],
 [4, [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]],
 [4, [28, 29, 30]],
 [4, [37, 38, 39]],
 [4, [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]],
 [4, [52, 53, 54]],
 [4, [55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]],
 [4, [67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78]],
 [4, [89, 90, 91, 92, 93, 94, 95, 96, 97, 98]],
 [4, [99, 100, 101, 102, 103, 104, 105, 106, 107, 108]],
 [4, [126]],
 [5, [3, 4, 5]],
 [5, [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]],
 [

In [8]:
student_classes = []

for s in students_id:
    student_classes.append([s, []])
    class_list = student_classes[-1][1]
    for i in students[s-1]['course']:
        for subpart in courses[i-1]['subpart']:
            for c in subpart['class']:
                class_list.append(c['id'])
student_classes

[[1, [1, 142, 143, 144, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]],
 [2,
  [1,
   142,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   113,
   114,
   115,
   116,
   117,
   118]],
 [3,
  [142,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108]],
 [4,
  [3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   

## SA pairs
SA_pairs are the pairs of classes that are constrained by the same attendee rule 

In [9]:
SA_pairs=[]
for ct in constraints:
    if ct['type']=='SameAttendees':
        for c1 in ct['class']:
            for c2 in ct['class']:
                if c1!=c2:
                    SA_pairs.append((c1,c2))

In [10]:
SA_pairs

[(1, 31),
 (1, 32),
 (1, 37),
 (1, 38),
 (1, 39),
 (31, 1),
 (31, 32),
 (31, 37),
 (31, 38),
 (31, 39),
 (32, 1),
 (32, 31),
 (32, 37),
 (32, 38),
 (32, 39),
 (37, 1),
 (37, 31),
 (37, 32),
 (37, 38),
 (37, 39),
 (38, 1),
 (38, 31),
 (38, 32),
 (38, 37),
 (38, 39),
 (39, 1),
 (39, 31),
 (39, 32),
 (39, 37),
 (39, 38),
 (3, 4),
 (3, 5),
 (3, 28),
 (3, 29),
 (3, 30),
 (4, 3),
 (4, 5),
 (4, 28),
 (4, 29),
 (4, 30),
 (5, 3),
 (5, 4),
 (5, 28),
 (5, 29),
 (5, 30),
 (28, 3),
 (28, 4),
 (28, 5),
 (28, 29),
 (28, 30),
 (29, 3),
 (29, 4),
 (29, 5),
 (29, 28),
 (29, 30),
 (30, 3),
 (30, 4),
 (30, 5),
 (30, 28),
 (30, 29),
 (40, 41),
 (40, 42),
 (40, 43),
 (40, 44),
 (40, 45),
 (40, 46),
 (40, 47),
 (40, 48),
 (40, 49),
 (40, 50),
 (40, 51),
 (41, 40),
 (41, 42),
 (41, 43),
 (41, 44),
 (41, 45),
 (41, 46),
 (41, 47),
 (41, 48),
 (41, 49),
 (41, 50),
 (41, 51),
 (42, 40),
 (42, 41),
 (42, 43),
 (42, 44),
 (42, 45),
 (42, 46),
 (42, 47),
 (42, 48),
 (42, 49),
 (42, 50),
 (42, 51),
 (43, 40),
 (43, 

# Check MIP Solution
- Change the sol file to solution you want to check as format sol_{dataset}_{approach}.py
while dataset is ['01', '04'] and approach is 'mp'

In [11]:
sol_file= "sol_01_mp.py"
fileObj=open(sol_file,"r")
content=fileObj.readlines()

##### extract x, y, z

In [12]:
x = []
for c in classes_id:
    x.append({'id':c})
    x[-1]['time'] = [0]*len(classes[c-1]['time'])    

y = []
for c in classes_id:
    y.append({'id':c})
    y[-1]['room'] = [0]*len(classes[c-1]['room'])      
    
z = [[0]*len(classes_id) for _ in range(len(students_id))]

In [13]:
y

[{'id': 1, 'room': [0, 0, 0]},
 {'id': 2, 'room': [0]},
 {'id': 3, 'room': [0]},
 {'id': 4, 'room': [0]},
 {'id': 5, 'room': [0]},
 {'id': 6, 'room': [0, 0, 0, 0]},
 {'id': 7, 'room': [0, 0, 0, 0]},
 {'id': 8, 'room': [0, 0, 0, 0]},
 {'id': 9, 'room': [0, 0, 0, 0]},
 {'id': 10, 'room': [0, 0, 0, 0]},
 {'id': 11, 'room': [0, 0, 0, 0]},
 {'id': 12, 'room': [0, 0, 0, 0]},
 {'id': 13, 'room': [0, 0, 0, 0]},
 {'id': 14, 'room': [0, 0, 0, 0]},
 {'id': 15, 'room': [0, 0, 0, 0]},
 {'id': 16, 'room': [0, 0, 0, 0]},
 {'id': 17, 'room': [0]},
 {'id': 18, 'room': [0]},
 {'id': 19, 'room': [0]},
 {'id': 20, 'room': [0]},
 {'id': 21, 'room': [0]},
 {'id': 22, 'room': [0]},
 {'id': 23, 'room': [0]},
 {'id': 24, 'room': [0]},
 {'id': 25, 'room': [0]},
 {'id': 26, 'room': [0]},
 {'id': 27, 'room': [0]},
 {'id': 28, 'room': [0]},
 {'id': 29, 'room': [0]},
 {'id': 30, 'room': [0]},
 {'id': 31, 'room': [0, 0, 0]},
 {'id': 32, 'room': [0, 0, 0]},
 {'id': 33, 'room': [0]},
 {'id': 34, 'room': [0]},
 {'id': 

In [14]:
for line in content[1:]:
    var, idx1, idx2 = line[:-3].split('_')
#     print(var, idx1, idx2)
    if var=='x':
        x[int(idx1)-1]['time'][int(idx2)] = 1
    elif var=='y':
        y[int(idx1)-1]['room'][int(idx2)] = 1
    elif var=='z':
        z[int(idx1)-1][int(idx2)-1] = 1

In [16]:
# check z
for s, classes in student_subparts:
    print(sum([z[s-1][c-1] for c in classes]))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [17]:
classes = []

for course in courses:
    for subpart in course['subpart']:
        for c in subpart['class']:
            classes.append(c)

classes
def get_class_by_id(classes, idx):
    for cl in classes:
        if cl["id"] == idx:
            return cl

# Convert MP solution to XML format

In [18]:
dataset_1 = "wbg-fal10"
dataset_4 = "pu-cs-fai07"

In [19]:
import xml.etree.cElementTree as ET

dataset=dataset_1
technique = "MIP"
author= "G1"
institution ="SMU"
root = ET.Element("solution", name = dataset, runtime="12.3", cores="4", technique=technique, author=author, institution=institution)

a=None
for cl in x:
    a = ET.SubElement(root, "class", id=str(cl["id"]))
    cl_needed = get_class_by_id(classes, cl["id"])
    for i in range(len(cl["time"])):
        if cl["time"][i] == 1:
            a.set('days', cl_needed["time"][i]["days"]) 
            a.set('start', str(cl_needed["time"][i]["start"]))
            a.set('weeks', cl_needed["time"][i]["weeks"])
    for rr in y:
        if rr["id"] == cl["id"]:
            for j in range(len(rr["room"])):
                if rr["room"][j] == 1:
                    a.set("room", str(cl_needed["room"][j]["id"]))
    for m in range(len(z)):
        if z[m][cl["id"]-1] ==1 :
            ET.SubElement(a, "student", id=str(m+1))
        
tree = ET.ElementTree(root)
tree.write(f"{dataset}-sol-mp.xml")
print("Done!")

Done!


# Check CP Solution

- Change the sol file to solution you want to check as format sol_{dataset}_{approach}.py while dataset is ['01', '04'] and approach is 'cp'



In [20]:
fileObj=open("sol_01_cp.py","r")
content=fileObj.readlines()

# Modeling

### Decision Variables

In [23]:
m = CpoModel('model1')

x = m.integer_var_list(len(classes_id), name='x')
y = m.integer_var_list(len(classes_id), name='y')
z = m.integer_var_dict([(s,sbp['id']) for s in students_id 
                        for i in students[s-1]['course'] 
                        for sbp in courses[i-1]['subpart']], 
                       name='z')

for c in classes_id:
    x[c-1].set_domain([i for i in range(0, len(classes[c-1]['time']))])
    y[c-1].set_domain([i for i in range(0, len(classes[c-1]['room']))])

for s in students_id:
    for i in students[s-1]['course']:
        for sbp in courses[i-1]['subpart']:
            z[s,sbp['id']].set_domain([c['id'] for c in sbp['class']])

In [25]:
z_list = [i for i in z]
z_list

[(1, 1),
 (1, 32),
 (1, 33),
 (1, 14),
 (1, 15),
 (2, 1),
 (2, 32),
 (2, 14),
 (2, 15),
 (2, 20),
 (2, 21),
 (3, 32),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 15),
 (3, 16),
 (3, 17),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 12),
 (4, 13),
 (4, 16),
 (4, 17),
 (4, 26),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 9),
 (5, 10),
 (5, 11),
 (5, 12),
 (5, 13),
 (5, 16),
 (5, 17),
 (5, 26),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 9),
 (6, 10),
 (6, 11),
 (6, 12),
 (6, 13),
 (6, 16),
 (6, 17),
 (6, 26),
 (7, 28),
 (7, 29),
 (7, 30),
 (7, 2),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 12),
 (7, 13),
 (7, 14),
 (7, 18),
 (7, 19),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 9),
 (8, 10),
 (8, 11),
 (8, 12),
 (8, 13),
 (8, 14),
 (8, 18),
 (8, 19),
 (9, 28),
 (9, 29),
 (9, 30),
 (9, 9),
 (9, 10),
 (9, 11),
 (9, 12),
 (9, 13),
 (9, 16),
 (9, 17),
 (9, 18),
 (9, 19),
 (10, 28),
 (10, 29),
 (10, 30),
 (10, 7),
 (10, 8),
 (10, 9),
 (10, 10),
 (10, 11),
 (10, 12),
 (10, 13),
 (10, 16),


# Set up cp variable to convert

In [26]:
x = []
for c in classes_id:
    x.append({'id':c})
    x[-1]['time'] = -1    

y = []
for c in classes_id:
    y.append({'id':c})
    y[-1]['room'] = -1     
    
z = [0 for i in z_list]

In [27]:
len(z_list)

199

In [28]:
for line in content[8:]:
    line =line.strip()
    n = line.find("=")
    var, idx1 = line[:n].split('_')
    val = line[n+1:]
    if var=='x':
        x[int(idx1)]['time'] = val
    elif var=='y':
        y[int(idx1)]['room'] = val
    elif var=='z':
        z[int(idx1)] = val

In [29]:
classes = []
for course in courses:
    for subpart in course['subpart']:
        for c in subpart['class']:
            c['sbp_id'] = subpart['id']
            classes.append(c)

classes
def get_class_by_id(classes, idx):
    for cl in classes:
        if cl["id"] == idx:
            return cl
classes

[{'id': 1,
  'limit': 2,
  'room': [{'id': 6, 'penalty': 4},
   {'id': 7, 'penalty': 0},
   {'id': 3, 'penalty': 0}],
  'time': [{'days': '1010100',
    'start': 102,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 16},
   {'days': '1010100',
    'start': 114,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 126,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 138,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 150,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 162,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 0},
   {'days': '1010100',
    'start': 174,
    'length': 10,
    'weeks': '1111111111111111',
    'penalty': 16},
   {'days': '1010100',
    'start': 186,
    'length': 10,
    'weeks': '1111111111111111',
   

In [30]:
sts = [[] for i in classes]
for m in range(len(z)):
    std_id = z_list[m][0]
    sb_id = z_list[m][1]
    cl_id = z[m]
    sts[int(cl_id)-1].append(std_id)
len(sts)

150

# Convert CP solution to XML format

In [31]:
import xml.etree.cElementTree as ET
dataset_2 = "lum-sum17"
dataset_1= "wbg-fal10"
dataset=dataset_1
technique = "MIP"
author= "G1"
institution ="SMU"
root = ET.Element("solution", name = dataset, runtime="12.3", cores="4", technique=technique, author=author, institution=institution)

a=None
for cl in x:
    a = ET.SubElement(root, "class", id=str(cl["id"]))
    cl_needed = get_class_by_id(classes, cl["id"])
#     for i in range(len(cl["time"])):
#         if cl["time"][i] == 1:
    a.set('days', cl_needed["time"][int(cl["time"])]["days"]) 
    a.set('start', str(cl_needed["time"][int(cl["time"])]["start"]))
    a.set('weeks', cl_needed["time"][int(cl["time"])]["weeks"])
    for rr in y:
        if rr["id"] == cl["id"]:
            a.set("room", str(cl_needed["room"][int(rr["room"])]["id"]))
    for i in range(len(sts)):
        if cl["id"]-1 == i:
            for s in sts[i]:
                ET.SubElement(a, "student", id=str(s))
        
tree = ET.ElementTree(root)
tree.write(f"{dataset}-sol-cp.xml")
print("Done!")

Done!
